In [93]:
import pandas as pd
import numpy as np
import sklearn as sk
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import alphalens
import warnings
warnings.filterwarnings("ignore")

Read data and see whether there is data missing.

In [2]:
data = pd.read_parquet("ohlcv_train_48sec.parquet.gzip")
print(data.info())
print(data.columns)
print(data.dtypes)
print(data.isna().sum())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21106332 entries, (Timestamp('2022-03-01 00:01:00+0000', tz='UTC'), 'asset_1') to (Timestamp('2022-05-27 00:00:00+0000', tz='UTC'), 'asset_3')
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Open    float64
 1   High    float64
 2   Low     float64
 3   Close   float64
 4   Volume  float64
dtypes: float64(5)
memory usage: 925.3+ MB
None
Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64
                                      Open    High     Low   Close    Volume
Close_time                Asset_id                                          
2022-03-01 00:01:00+00:00 asset_1   1.5031  1.5044  1.5004  1.5007   27197.0
2022-03-01 00:02:00+00:00 asset_1   1.5007  1.5051  1.4999  1.5044   40074.0
2022-03-01 00:03:00+00:00 ass

Reset the multi indexes and see the data again

In [3]:
df = data.copy()
df = df.reset_index()
#datetime64[us, UTC]
print(df.info())
print(df.columns)
print(df.dtypes)
print(df.isna().sum())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21106332 entries, 0 to 21106331
Data columns (total 7 columns):
 #   Column      Dtype              
---  ------      -----              
 0   Close_time  datetime64[us, UTC]
 1   Asset_id    object             
 2   Open        float64            
 3   High        float64            
 4   Low         float64            
 5   Close       float64            
 6   Volume      float64            
dtypes: datetime64[us, UTC](1), float64(5), object(1)
memory usage: 1.1+ GB
None
Index(['Close_time', 'Asset_id', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Close_time    datetime64[us, UTC]
Asset_id                   object
Open                      float64
High                      float64
Low                       float64
Close                     float64
Volume                    float64
dtype: object
Close_time    0
Asset_id      0
Open          0
High          0
Low           0
Close         0
Volume        0
dtype: int64
   

Task 1: Return the 5 min return (rolling)

In [4]:
group = df.groupby('Asset_id')
output_5min_return_rate = pd.DataFrame()
for asset, info in group:
    info.set_index(['Close_time'], inplace=True)
    info['Last_Close'] = info['Close'].shift(1)
    close_1 = info['Last_Close'].pct_change()
    close_1.fillna(0,inplace=True)
    close_5 = (close_1+1).rolling(window=5, min_periods=1).apply(np.prod, raw=True)-1
    #print(type(close_5)) #<class 'pandas.core.series.Series'>

    output_5min_return_rate.insert(output_5min_return_rate.shape[1], asset, close_5)  
print(output_5min_return_rate) # task 1 ourput
#output_5min_return_rate.to_csv("Answer/Task 1 Answer.csv", mode='w')

                            asset_1  asset_10  asset_11  asset_12  \
Close_time                                                          
2022-03-01 00:01:00+00:00  0.000000  0.000000  0.000000  0.000000   
2022-03-01 00:02:00+00:00  0.000000  0.000000  0.000000  0.000000   
2022-03-01 00:03:00+00:00  0.002466  0.000910  0.001710  0.002089   
2022-03-01 00:04:00+00:00  0.008329  0.004725  0.007160  0.008156   
2022-03-01 00:05:00+00:00  0.005264  0.002603  0.003282  0.004874   
...                             ...       ...       ...       ...   
2022-12-31 23:55:00+00:00 -0.001043 -0.002271 -0.000127 -0.000953   
2022-12-31 23:56:00+00:00  0.000000 -0.001461  0.000151  0.000000   
2022-12-31 23:57:00+00:00 -0.000261 -0.000853  0.000103  0.000715   
2022-12-31 23:58:00+00:00 -0.000261 -0.000771  0.000224  0.000954   
2022-12-31 23:59:00+00:00 -0.000522 -0.000975  0.000091  0.000716   

                               asset_13  asset_14  asset_15  asset_16  \
Close_time                   

Task 2: Data Preprocessing with standard function

In [5]:
# for continuous variables

# drop columns missing rate above threshold
def drop(data,threshold):
    data = data[data.columns[data.isnull().mean() < threshold]]
    
# replace missing value with 0
def replace0(data):
    data = data.fillna(0)
# replace missing value with the most frequent value of each column
def replace0(data):
    #Categorical imputation
    for i in list(data):
        data[i].fillna(data[i].value_counts().idxmax(), inplace=True)
# replace extreme large/small values with upper/lower bound
def extreme_MAD(data, threshold):
    data = standardize(data)
    median = data.quantile(0.5)
    new_median = (abs(data-median)).quantile(0.5)
    upper_bound = median + threshold * new_median
    lower_bound = median - threshold * new_median
    return data.clip(lower_bound, upper_bound, axis=1)
        
def identify_outliers(data, threshold):
    data = standardize(data)
    for i in list(data):
        outliers = [x for x in data[i] if abs(x) > threshold]
    print('%s has outliers (out of %s standard deviation): %s' %(i, threshold, len(outliers)))
    
    
# frequency bar plot to check normal distribution
def return_distribution(data):
    list_bin = [-6,-5.5,-5,-4.5,-4,-3.5,-3,-2.5,-2,-1.5,-1,-0.5,0,0.5,1.0,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6]
    list_label = [ u"(-6,-5.5]", u"(-5.5,-5]", u"(-5,-4.5]", u"(-4.5,-4]", u"(-4,-3.5]", u"(-3.5,-3]", u"(-3,-2.5]", u"(-2.5,-2]", "u(-2,-1.5]", u"(-1.5,-1]", u"(-1,-0.5]", 
    u"(-0.5,0]", u"(0,0.5]", u"(0.5,1]", u"(1,1.5]", u"(1.5,2]", u"(2,2.5]", u"(2.5,3]", u"(3,3.5]", u"(3.5,4]", u"(4,4.5]", u"(4.5,5)", u"(5,5.5]", u"(5.5,6)"]
    # for i in list(data):
    #     #sns.boxplot(x=data[i])
    #     data[i].plot(kind='line')
    data = standardize(data)
    for i in list(data):
        a = pd.cut(data[i], list_bin, labels=list_label)
        b = a.value_counts()
        b1 = b.sort_index()
        c = {'section': b1.index, 'frequency': b1.values}
        e = pd.DataFrame(c)
        ax = plt.figure(figsize=(20,15)).add_subplot(111)
        sns.barplot(x='section', y='frequency', data = e)
        #ax.set_ylim([0,30])
        ax.set_xlabel('z-score interval', fontsize=20)
        ax.set_ylabel('frequency', fontsize=20)
        ax.set_title('frequency plot of %s'%i, size=40)
        
        for x,y in zip(range(len(e)), e.frequency):
            ax.text(x,y, '%d'%y, ha='center', va='bottom', fontsize=30, color='grey')
        #plt.savefig('/figure/frequency plot of %s'%i, bbox_inches ="tight")

# box_plots to check out liner point
def box_plot(data):
    for i in list(data):
        sns.boxplot(x=data[i])
       
# non-dimensionalization to 0-1
def zero_to_one(data):
    output_01 = pd.DataFrame()
    for i in list(data):
        output_01[i] = (data[i] - np.min(data[i]) ) / (np.max(data[i]) - np.min(data[i]) )
    return output_01

# normalization to z-score
def standardize(data):
    output_standard = pd.DataFrame()
    for i in list(data):
        output_standard[i] = (data[i] - np.mean(data[i])) / np.std(data[i])
    return output_standard
        
#print(output_5min_return_rate.head())
#print(zero_to_one(output_5min_return_rate).head())
#print(standardize(output_5min_return_rate).head())
#return_distribution(output_5min_return_rate)
#box_plot(standardize(output_5min_return_rate))
#return_distribution(extreme_MAD(output_5min_return_rate,12))

Task 3: IC (Information Coeffieient), IR (Information Ratio)
Pearson Correlation: suit for normal distribution, linear 
Spearman Correlation: suit for non-normal distribuition, nonlinear, robust

Normal IC: $IC_{i}$ = Pearson(predict return calculated by $factor_i$ at time t-1, actural return at time t)

Rank IC: $Rank~IC_{i}$ = Spearman(the rank of predict return calculated by $factor_i$ at time t, the rank of actural return at time t+1)

IR: Average Rank IC among different time period/ standard deviation of Rank IC among different time period

$\rho \left( r^{t},f^{t-1} \right) 
= \frac{Cov\left( r^{t},f^{t-1}\right)} {\sqrt{Var\left(r^{t}\right)} \cdot \sqrt{Var\left(f^{t-1} \right)}}
=\frac{\sum\limits_{i=1}^{N}{\left( r_{i}^{t}-\bar{r}^{t} \right)\left( f_{i}^{t-1}-\bar{f}^{t-1} \right)}}{\sqrt{\sum\limits_{i=1}^{N}{{{\left( r_{i}^{t}-\bar{r}^{t} \right)}^{2}}}\cdot \sum\limits_{i=1}^{N}{{{\left( f_{i}^{t-1}-\bar{f}^{t-1} \right)}^{2}}}}}$

In [95]:
period_list = [1, 2, 3, 7, 15, 30]
factor_list = ['MA','Momentum_Close','Momentum_Volume','std_Close_to_Open', 'corr_Close_to_Volume']

def cal_factor(df,factor_name):
    if factor_name == 'MA':
        df['MA'] = df['Close'].shift(1).rolling(window=5, min_periods=1).mean()
        df['MA'] = df['MA'].fillna(0)
        # return df['MA']
    if factor_name == 'Momentum_Close':
        # Momentum
        df['Momentum_Close'] = (df['Close'].shift(1) - df['Close'].shift(2))/df['Close'].shift(2)
        df['Momentum_Close'] = df['Momentum_Close'].fillna(0)
        # return df['Momentum']
    if factor_name == 'Momentum_Volume':
        df['Momentum_Volume'] = (df['Volume'].shift(1) - df['Volume'].shift(2))/df['Volume'].shift(2)
        df['Momentum_Volume'] = df['Momentum_Volume'].fillna(0)
        # return df['open_volume']
    if factor_name == 'std_Close_to_Open':
        df['std_Close_to_Open'] = (df['Close']/df['Open']).shift(1).std()
        df['std_Close_to_Open'] = df['std_Close_to_Open'].fillna(0)
    if factor_name == 'corr_Close_to_Volume':
        df['corr_Close_to_Volume'] = df['Close'].shift(5).corr(df['Volume'].shift(1))
        df['corr_Close_to_Volume'] = df['corr_Close_to_Volume'].fillna(0)
    if factor_name == 'Volume':
        df['Volume_1'] = df['Volume'].shift(1)
        df['Volume_1'] = df['Volume'].fillna(0)
        # return df['Volume']
    

def IC_data(period, df, factor_name):  
    group = df.groupby("Asset_id")
    day_return_rate = pd.DataFrame()
    factor = pd.DataFrame()
   
    for asset, info in group:
        ohlcv_dict = {
            'Open':'first',
            'High':'max',
            'Low':'min',
            'Close':'last',
            'Volume':'sum'
        }
        info.set_index(['Close_time'], inplace=True)
        info = info.resample('1d').agg(ohlcv_dict)
        #print(info)
        daily = info.copy()
        daily = daily.reset_index(drop=False)
        daily['Close_time'] = pd.DatetimeIndex(daily['Close_time']).date
        daily = daily.set_index('Close_time')
        daily['Asset_id'] = asset

        
        daily['Close_period'] = daily['Close'].shift(int(period))
        close_period = daily['Close_period'].pct_change()
        close_period.fillna(0,inplace=True) # <class 'pandas.core.series.Series'>
        
        cal_factor(daily,factor_name)
        #print(daily)
        factor_value = daily.iloc[:,-1]
    
        day_return_rate.insert(day_return_rate.shape[1], asset, close_period) 
        
        factor.insert(factor.shape[1], asset, factor_value)
            
    return(day_return_rate, factor)


In [96]:
#alphalens is out of maintain. I can't call it anyway.
# factor_return = alphalens.utils.get_clean_factor_and_forward_returns(factor_alphalens['Volume'].shift(48), price_alphalens, quantiles=5, periods=(1,5,10))

def normal_IC(day_return_rate, factor):
    df = pd.DataFrame()
    temp = day_return_rate.copy()
    temp = temp.reset_index(drop=False)
    df['Close_time'] = temp['Close_time']
    #df.set_index('Close_time', inplace=True)
    df['IC'] = 0
    for i in range(day_return_rate.shape[0]):   
        df.loc[i,'IC'] = np.corrcoef(np.asarray(factor.iloc[i,:][~np.isnan(factor.iloc[i,:])]), np.asarray(day_return_rate.iloc[i,:][~np.isnan(day_return_rate.iloc[i,:])]))[1][0]     
    df = df.set_index(['Close_time'])
    return df


def rank_IC(day_return_rate, factor):
    df = pd.DataFrame()
    temp = day_return_rate.copy()
    temp = temp.reset_index(drop=False)
    df['Close_time'] = temp['Close_time']
    df['Rank_IC'] = 0
    for i in range(day_return_rate.shape[0]):
        # print(np.asarray(factor.iloc[i,:][~np.isnan(factor.iloc[i,:])]))
        # print(np.asarray(day_return_rate.iloc[i,:][~np.isnan(day_return_rate.iloc[i,:])]))

        df.loc[i,'Rank_IC'] = stats.pearsonr(np.asarray(factor.iloc[i,:][~np.isnan(factor.iloc[i,:])]), np.asarray(day_return_rate.iloc[i,:][~np.isnan(day_return_rate.iloc[i,:])])).statistic
        # np.isnan    pd.isnull
    df = df.set_index(['Close_time'])
    return df

period_list = [1, 2, 3, 7, 15, 30]
def IR(period_list, factor_name):
    muti_IC = pd.DataFrame()
    for period in period_list:
        day_return_rate, factor = IC_data(period, input, factor_name)
        if period == 1:
            temp = day_return_rate.copy()
            temp = temp.reset_index(drop=False)
            muti_IC['Close_time'] = temp['Close_time']
            muti_IC = muti_IC.set_index('Close_time', drop=True)  
        temp = rank_IC(day_return_rate, factor).copy()
        temp = temp['Rank_IC']
        #muti_IC = muti_IC.insert(muti_IC.shape[1], period, temp)
        muti_IC = pd.merge(muti_IC, temp, on='Close_time')
        muti_IC.rename(columns={'Rank_IC':'%s_days'%period}, inplace=True)
    return muti_IC
input = data.copy()
input = df.reset_index()
muti_IC = IR(period_list, factor_name='Volume')

In [97]:
print(muti_IC)
a = muti_IC.mean(axis=0)
b = muti_IC.std(axis=0)
print('各周期RankIC均值: %s' %a)
print('各周期RankIC标准差: %s' %b)
c = a.mean()
d = a.std()
e = c/d
print('IC:%s, IR:%s'%(c,e))

              1_days    2_days    3_days    7_days   15_days   30_days
Close_time                                                            
2022-03-01       NaN       NaN       NaN       NaN       NaN       NaN
2022-03-02       NaN       NaN       NaN       NaN       NaN       NaN
2022-03-03 -0.179515       NaN       NaN       NaN       NaN       NaN
2022-03-04 -0.059367 -0.178987       NaN       NaN       NaN       NaN
2022-03-05  0.084681 -0.051269 -0.178502       NaN       NaN       NaN
...              ...       ...       ...       ...       ...       ...
2022-12-27 -0.069254 -0.246634  0.036013 -0.019574 -0.032484  0.497996
2022-12-28 -0.086496 -0.119359 -0.236649 -0.264511  0.000152 -0.075546
2022-12-29 -0.017240 -0.058493 -0.086018  0.047076 -0.167681 -0.011484
2022-12-30  0.154508 -0.043678 -0.107260 -0.091251  0.086178 -0.116588
2022-12-31 -0.032171  0.075289 -0.057344 -0.033533 -0.003837 -0.294316

[306 rows x 6 columns]
各周期RankIC均值: 1_days     0.054956
2_days     0.026607


Task 4

https://www.ricequant.com/doc/rqdata/python/factors-dictionary.html#%E5%9D%87%E7%BA%BF%E7%B1%BB%E6%8C%87%E6%A0%87

Size
Momentum 

(Return_rate_1day, Return_rate_3day, Return_rate_7day, Return_rate_15day)

Volume 

(log_volume, log_price*volume, log_price*volume/turnover_rate, )

Volatility

(std_close_open)

In [98]:
period_list = [1, 2, 3, 7, 15, 30]
factor_list = ['MA','Momentum_Close','Momentum_Volume','std_Close_to_Open', 'corr_Close_to_Volume']

for factor_name in factor_list:

    muti_IC = IR(period_list, factor_name)
    #print(muti_IC)
    a = muti_IC.mean(axis=0)
    b = muti_IC.std(axis=0)
    c = a.mean()
    d = a.std()
    e = c/d
    print('factor_%s ,IC:%s, IR:%s'%(factor_name,c,e))




factor_MA ,IC:0.00976411285335221, IR:5.939543849763861
factor_Momentum_Close ,IC:0.16112571807109644, IR:0.39177783379310355
factor_Momentum_Volume ,IC:0.05181719822092463, IR:0.26539625806273426
factor_std_Close_to_Open ,IC:-0.022892180404016072, IR:-4.113783399588695
factor_corr_Close_to_Volume ,IC:0.0017140867742168034, IR:0.7972066113588716


Appendix

有待改进：
1 没有找到rank_IC > 0.05 且 IR > 0.5的因子。可以考虑采用多因子模型，主成分分析法，筛选出一系列得分高且不太相关的因子 进行组合。
2 代码框架乱，函数封装得不好

Resampled data

In [ ]:
# path2 = 'data5min'
# os.makedirs(path2, exist_ok=True)  
# files1 = [f for f in os.listdir(path1) if f.endswith('.csv')]
# for file in files1:
#     file_path = os.path.join(path1, file)
#     df = pd.read_csv(file_path, parse_dates=['Close_time'], index_col=['Close_time'])
#     
#     df['Open'] = df['Open'].resample('5min').first()
#     df['High'] = df['High'].resample('5min').max()
#     df['Low'] = df['Low'].resample('5min').min()
#     df['Close'] = df['Close'].resample('5min').last()
#     df['Volume'] = df['Volume'].resample('5min').sum()
#     df = df.resample('5min').last()
#     asset = df['Asset_id'].iloc[0]
#     # print(df)
#     # print(asset)
#     # print(df.head())
#     # print(df.tail())
#     df.to_csv("%s/%s.csv"% (path2,asset), index=True, mode='w')
#     

In [ ]:
# files2 = [f for f in os.listdir(path2) if f.endswith('.csv')]
# for file in files2:
#     file_path = os.path.join(path2, file)
#     df = pd.read_csv(file_path, parse_dates=['Close_time'], index_col=['Close_time'])
#     df['LastClose'] = df['Close'].shift(1)
#     # num = float(df['Open'].iloc[0].copy())
#     # df.loc[0, "LastClose"] = num
#     # df['LastClose'].iloc[0] = df['Open'].iloc[0]
#     close_5 = df['LastClose'].pct_change()
#     close_5.fillna(0,inplace=True)
#     print(close_5.head())
# 
#     #df.to_csv("%s/%s.csv"% (path2,asset), index=True, mode='w')

In [ ]:
IC rankIC IR

In [ ]:
# group = df.groupby("Asset_id")
# day_return_rate = pd.DataFrame()
# factor = pd.DataFrame()
# factor_alphalens = pd.DataFrame()
# price_alphalens = pd.DataFrame()
# #period_list = {1, 2, 3, 7, 15, 30}
# 
# for asset, info in group:
#     ohlcv_dict = {
#         'Open':'first',
#         'High':'max',
#         'Low':'min',
#         'Close':'last',
#         'Volume':'sum'
#     }
#     info.set_index(['Close_time'], inplace=True)
#     info = info.resample('1d').agg(ohlcv_dict)
#     #print(info)
#     daily = info.copy()
#     daily = daily.reset_index(drop=False)
#     daily['Close_time'] = pd.DatetimeIndex(daily['Close_time']).date
#     daily = daily.set_index('Close_time')
#     daily['Asset_id'] = asset
#     #print(daily)
# 
# 
#     daily['Last_Close'] = daily['Close'].shift(1)
#     close_period = daily['Close'].pct_change()
#     close_period.fillna(0,inplace=True) # <class 'pandas.core.series.Series'>
# 
#     close = daily['Close'] # <class 'pandas.core.series.Series'>
#     factor_value = daily['Volume']
# 
#     day_return_rate.insert(day_return_rate.shape[1], asset, close_period) 
# 
#     factor.insert(factor.shape[1], asset, factor_value)
# 
#     factor_alphalens = pd.concat([factor_alphalens, daily[['Asset_id','Volume']]], axis=0)
# 
#     price_alphalens.insert(price_alphalens.shape[1], asset, close)
# #
# factor_alphalens = factor_alphalens.reset_index(drop=False)
# factor_alphalens.index = pd.to_datetime(factor_alphalens['Close_time'])
# factor_alphalens = factor_alphalens.drop(factor_alphalens.columns[[0]], axis=1)
# factor_alphalens.index.name = None
# factor_alphalens.sort_index(inplace=True)
# factor_alphalens = factor_alphalens.set_index([factor_alphalens.index, factor_alphalens['Asset_id']], drop = True)
# factor_alphalens = factor_alphalens.drop(factor_alphalens.columns[[0]], axis=1)
# factor_alphalens.sort_index(inplace=True)
# factor_alphalens.index.name = None
# 
# 
# # factor_alphalens = factor_alphalens.groupby(['Close_time', 'Asset_id']).mean()
# 
# # factor_alphalens = factor_alphalens.reset_index(drop=False)
# # factor_alphalens = factor_alphalens.groupby(['Close_time']).apply(lambda x: x[:])
# # factor_alphalens = factor_alphalens.drop(factor_alphalens.columns[[0]], axis=1)
# # factor_alphalens = factor_alphalens.reset_index(drop=False)
# # factor_alphalens = factor_alphalens.drop(factor_alphalens.columns[[1]], axis=1)
# # factor_alphalens = factor_alphalens.set_index(['Close_time', 'Asset_id'], inplace =False)
# 
# print(day_return_rate) 
# print(factor)
# 
# # print(factor_alphalens)
# # print(price_alphalens)